In [3]:
import numpy as np
import scipy
import skimage
import sklearn
import cv2
from skimage.measure import label
from skimage.morphology import remove_small_objects
from skimage.feature import match_template
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imread
from skimage import color

In [4]:
def convolve(image, kernel, normed=False):
    assert image.ndim == kernel.ndim == 2
    shape = np.array(image.shape) - kernel.shape + 1
    result = np.empty(shape, dtype=float)
    h, w = kernel.shape

    if normed:
        kernel_normed = kernel - np.mean(kernel)
        k_norm = np.sum(kernel_normed ** 2)

    for i, j in np.ndindex(*shape):
        if normed:
            patch = image[i:i + h, j:j + w]
            patch_normed = patch - np.mean(patch)
            result[i, j] = np.sum(patch_normed * kernel_normed) / np.sqrt(np.sum(patch_normed ** 2) * k_norm)
        else:
            result[i, j] = (image[i:i + h, j:j + w] * kernel).sum()

    return result

    
def convolve_same(image, kernel, normed=False):
    delta = np.array(kernel.shape) - 1
    left = delta // 2
    right = delta - left
    padding = np.array([left, right]).T

    image = np.pad(image, padding, mode='constant') 
    return convolve(image, kernel, normed=normed)

def get_local_centers(corr, th):
    lbl, n = label(corr >= th, connectivity=2, return_num=True)
    return np.int16([np.round(np.mean(np.argwhere(lbl == i), axis=0)) for i in range(1, n + 1)])
def find_centers(img):
    img_rgb = img[..., ::-1]
    chanel = img_rgb[:,:,0]
    res = cv2.resize(chanel, (int(chanel.shape[1]/3), int(chanel.shape[0]/3)), interpolation= cv2.INTER_LINEAR)
    template = np.asarray([[141, 188, 124, 140, 174, 174, 165, 149, 124, 171, 152], [161, 115, 182, 199, 210, 197, 203, 204, 202, 135, 167], [91, 191, 213, 254, 187, 156, 144, 171, 219, 194, 146], [125, 156, 244, 247, 198, 189, 172, 154, 184, 214, 169], [108, 158, 211, 200, 203, 200, 196, 175, 149, 207, 175], [95, 157, 204, 201, 201, 199, 196, 194, 160, 180, 186], [93, 142, 202, 199, 199, 198, 196, 197, 178, 176, 170], [92, 95, 191, 191, 192, 179, 193, 198, 178, 184, 130], [77, 89, 98, 180, 182, 173, 177, 178, 180, 161, 108], [82, 85, 92, 95, 133, 146, 155, 160, 156, 122, 143], [70, 74, 86, 95, 96, 114, 119, 115, 96, 164, 175]])
    corr_conv = convolve_same(res, template, normed=True)
    centers = get_local_centers(corr_conv, 0.78)
    centers = centers * 3
    return(centers)

def red(img):
    dim = (800, 600)
    img = cv2.resize(img, dim)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    lab = rgb2lab(img_rgb)
    mask = lab[:, :, 1]>39
    mask = remove_small_objects(mask, min_size = 150)
    mask = mask.astype(np.uint8)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    img_rgb_contours = img_rgb.copy()
    cv2.drawContours(img_rgb_contours, contours, -1, (255, 255, 255), 1)

    img_rgb_contours_centers = img_rgb_contours.copy()
    centers_coords_list = []
    area = []
    number = 0
    for i in range(len(contours)): 
       cont = contours[i]
       area.append([cv2.contourArea(cont)])
    area = np.array(area)
    for i in range(len(contours)):
        if area[i]>20:
            number = number + 1
            cnt = contours[i]
            m = cv2.moments(cnt)
            cx = int(m['m10']/m['m00'])
            cy = int(m['m01']/m['m00'])
            centers_coords_list.append([cx, cy])
            cv2.circle(img_rgb_contours_centers, (cx, cy), 2, (0, 255, 0), 2)
    return(number)

def green(img):
    dim = (800, 600)
    img = cv2.resize(img, dim)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    mask = mask[:, :, 1]
    mask = (mask > 90) & (mask < 104)
    mask = remove_small_objects(mask, min_size = 50)
    mask = mask.astype(np.uint8)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    img_rgb_contours = img_rgb.copy()
    cv2.drawContours(img_rgb_contours, contours, -1, (255, 255, 255), 1)
    img_rgb_contours_centers = img_rgb_contours.copy()
    centers_coords_list = []
    area = []
    number = 0
    for i in range(len(contours)): 
       cont = contours[i]
       area.append([cv2.contourArea(cont)])
    area = np.array(area)
    for i in range(len(contours)):
        if area[i]>20:
            number = number + 1
            cnt = contours[i]
            m = cv2.moments(cnt)
            cx = int(m['m10']/m['m00'])
            cy = int(m['m01']/m['m00'])
            centers_coords_list.append([cx, cy])
            cv2.circle(img_rgb_contours_centers, (cx, cy), 2, (0, 255, 0), 2)
        return(number)

def blue(img):
    dim = (800, 600)
    img = cv2.resize(img, dim)
    img_rgb_contours = img

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )

    h_min = np.array((99, 175, 75), np.uint8)
    h_max = np.array((104, 255, 148), np.uint8)

    thresh = cv2.inRange(hsv, h_min, h_max)
    thresh = cv2.bitwise_not(thresh)
    mask = remove_small_objects(thresh, min_size = 4500)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img_rgb_contours, contours, -1, (255, 255, 255), 1)

    img_rgb_contours_centers = img_rgb_contours.copy()
    centers_coords_list = []
    area = []
    number = 0
    for i in range(len(contours)): 
       cont = contours[i]
       area.append([cv2.contourArea(cont)])
    area = np.array(area)
    for i in range(len(contours)):
        if area[i]>50:
            number = number + 1
            cnt = contours[i]
            m = cv2.moments(cnt)
            cx = int(m['m10']/m['m00'])
            cy = int(m['m01']/m['m00'])
            centers_coords_list.append([cx, cy])
            cv2.circle(img_rgb_contours_centers, (cx, cy), 2, (0, 255, 0), 2)
    return(number)

def black(img):
    dim = (800, 600)
    img = cv2.resize(img, dim)
    img_rgb_contours = img

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )

    h_min = np.array((0, 0, 47), np.uint8)
    h_max = np.array((255, 255, 255), np.uint8)

    thresh = cv2.inRange(hsv, h_min, h_max)
    thresh = cv2.bitwise_not(thresh)
    mask = remove_small_objects(thresh, min_size = 17000)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img_rgb_contours, contours, -1, (255, 255, 255), 1)

    img_rgb_contours_centers = img_rgb_contours.copy()
    centers_coords_list = []
    area = []
    number = 0
    for i in range(len(contours)): 
       cont = contours[i]
       area.append([cv2.contourArea(cont)])
    area = np.array(area)
    for i in range(len(contours)):
        if area[i]>50:
            number = number + 1
            cnt = contours[i]
            m = cv2.moments(cnt)
            cx = int(m['m10']/m['m00'])
            cy = int(m['m01']/m['m00'])
            centers_coords_list.append([cx, cy])
            cv2.circle(img_rgb_contours_centers, (cx, cy), 2, (0, 255, 0), 2)
    return(number)

def yellow(img):
    dim = (800, 600)
    img = cv2.resize(img, dim)
    img_rgb_contours = img

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )

    h_min = np.array((21, 158, 189), np.uint8)
    h_max = np.array((139, 255, 255), np.uint8)

    thresh = cv2.inRange(hsv, h_min, h_max)
    mask = remove_small_objects(thresh, min_size = 200)

    mask = mask.astype(np.uint8)


    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img_rgb_contours, contours, -1, (255, 255, 255), 1)


    img_rgb_contours_centers = img_rgb_contours.copy()
    centers_coords_list = []
    area = []
    number = 0
    for i in range(len(contours)): 
       cont = contours[i]
       area.append([cv2.contourArea(cont)])
    area = np.array(area)
    for i in range(len(contours)):
        if area[i]>50:
            number = number + 1
            cnt = contours[i]
            m = cv2.moments(cnt)
            cx = int(m['m10']/m['m00'])
            cy = int(m['m01']/m['m00'])
            centers_coords_list.append([cx, cy])
            cv2.circle(img_rgb_contours_centers, (cx, cy), 2, (0, 255, 0), 2)
    return(number)

In [5]:
def predict_image(img):
    centers = find_centers(img)
    n_trains = {"blue": blue(img), "green": green(img), "black": black(img), "yellow": yellow(img), "red": red(img)}
    scores = {"blue": blue(img) * 1.45, "green": green(img)* 1.45, "black": black(img)* 1.45, "yellow": yellow(img)*1.45, "red": red(img)*1.45}
    return(centers, n_trains, scores)
